In [23]:
# ============================
# Call 1: ตั้งค่าเริ่มต้น + import + โหลด meta (multitask dataset)
# ============================

import os
import json
import numpy as np
import pandas as pd
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, Tuple, List

import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# ---- device ----
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print("Device:", device)

# ---- paths (multitask dataset) ----
DATA_DIR = Path(
    "/Users/thanaporn/Desktop/EURO_H1_AI/prepared_datasets/boosting_dl_multitask"
)
NPZ_PATH = DATA_DIR / "eurusd_multitask_sequences.npz"
META_PATH = DATA_DIR / "eurusd_multitask_meta.json"

with open(META_PATH, "r", encoding="utf-8") as f:
    meta = json.load(f)

print("Loaded meta keys:", meta.keys())
print("CSV path:", meta["csv_path"])
print("Seq len:", meta["seq_len"], " Horizon:", meta["horizon"])
print("Feature cols:", len(meta["feature_cols"]))
print("Targets boosting:", meta["targets_boosting"])
print("Targets DL:", meta["targets_dl"])


# ใช้ค่าใน meta เป็น config สำหรับสคริปต์นี้
@dataclass
class TrainConfig:
    csv_path: str
    seq_len: int
    horizon: int
    train_ratio: float
    val_ratio: float
    feature_cols: List[str]
    targets_boosting: List[str]
    targets_dl: List[str]


train_cfg = TrainConfig(
    csv_path=meta["csv_path"],
    seq_len=meta["seq_len"],
    horizon=meta["horizon"],
    train_ratio=meta["train_ratio"],
    val_ratio=meta["val_ratio"],
    feature_cols=meta["feature_cols"],
    targets_boosting=meta["targets_boosting"],
    targets_dl=meta["targets_dl"],
)

feature_cols = train_cfg.feature_cols
targets_boosting = train_cfg.targets_boosting
targets_dl = train_cfg.targets_dl

Device: mps
Loaded meta keys: dict_keys(['csv_path', 'seq_len', 'horizon', 'train_ratio', 'val_ratio', 'feature_cols', 'columns_required', 'targets_boosting', 'targets_dl', 'note'])
CSV path: data_csv/EURUSD_D1.csv
Seq len: 51  Horizon: 1
Feature cols: 24
Targets boosting: ['tgt_high_break', 'tgt_low_break', 'tgt_vol_high', 'tgt_candle_class']
Targets DL: ['tgt_high_break', 'tgt_low_break', 'tgt_vol_high', 'tgt_candle_class']


In [24]:
# ============================
# Call 2: โหลด NPZ dataset (multitask) ให้พร้อมเทรน
# ============================

npz = np.load(NPZ_PATH, allow_pickle=True)

# ---- Boosting tabular ----
Xb_train = npz["Xb_train"].astype(np.float32)
Xb_val = npz["Xb_val"].astype(np.float32)
Xb_test = npz["Xb_test"].astype(np.float32)

yb_train = npz["yb_train"].astype(np.int64)  # [N, 4] = 4 targets multitask
yb_val = npz["yb_val"].astype(np.int64)
yb_test = npz["yb_test"].astype(np.int64)

idxb_train = pd.to_datetime(npz["idxb_train"])
idxb_val = pd.to_datetime(npz["idxb_val"])
idxb_test = pd.to_datetime(npz["idxb_test"])

# ---- DL sequences (multi-task targets) ----
Xs_train = npz["Xs_train"].astype(np.float32)  # [N, seq_len, F]
Xs_val = npz["Xs_val"].astype(np.float32)
Xs_test = npz["Xs_test"].astype(np.float32)

yd_train = npz["yd_train"].astype(np.int64)  # [N, 4] = 4 targets multitask
yd_val = npz["yd_val"].astype(np.int64)
yd_test = npz["yd_test"].astype(np.int64)

idxs_train = pd.to_datetime(npz["idxs_train"])
idxs_val = pd.to_datetime(npz["idxs_val"])
idxs_test = pd.to_datetime(npz["idxs_test"])

# ---- ตรวจสอบให้ตรงกับ meta (Call 1) ----
print("Boosting train:", Xb_train.shape, yb_train.shape)
print("Boosting val:  ", Xb_val.shape, yb_val.shape)
print("Boosting test: ", Xb_test.shape, yb_test.shape)

print("DL train:", Xs_train.shape, yd_train.shape)
print("DL val:  ", Xs_val.shape, yd_val.shape)
print("DL test: ", Xs_test.shape, yd_test.shape)

print("Num features (tabular):", Xb_train.shape[1], " / meta:", len(feature_cols))
print("Num targets (boosting):", yb_train.shape[1], " / meta:", len(targets_boosting))
print("Num targets (DL):      ", yd_train.shape[1], " / meta:", len(targets_dl))

Boosting train: (2072, 24) (2072, 4)
Boosting val:   (444, 24) (444, 4)
Boosting test:  (444, 24) (444, 4)
DL train: (2036, 51, 24) (2036, 4)
DL val:   (436, 51, 24) (436, 4)
DL test:  (437, 51, 24) (437, 4)
Num features (tabular): 24  / meta: 24
Num targets (boosting): 4  / meta: 4
Num targets (DL):       4  / meta: 4


In [25]:
# ============================
# Call 3: Train Boosting (LightGBM classifiers) + ทำ pred ทั้งชุด
# ============================

from sklearn.metrics import accuracy_score, f1_score

target_names = targets_boosting  # ["tgt_high_break","tgt_low_break","tgt_vol_high","tgt_candle_class"]

params_common = dict(
    n_estimators=1500,
    learning_rate=0.02,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    random_state=42,
)

boost_models = {}


def train_one_target(i: int, name: str):
    if name == "tgt_candle_class":
        params = dict(objective="multiclass", num_class=3, **params_common)
        model = lgb.LGBMClassifier(**params)
        eval_metric = "multi_logloss"
    else:
        params = dict(objective="binary", **params_common)
        model = lgb.LGBMClassifier(**params)
        eval_metric = "binary_logloss"

    model.fit(
        Xb_train,
        yb_train[:, i],
        eval_set=[(Xb_val, yb_val[:, i])],
        eval_metric=eval_metric,
        callbacks=[lgb.early_stopping(200, verbose=False)],
    )
    return model


for i, tname in enumerate(target_names):
    m = train_one_target(i, tname)
    boost_models[tname] = m
    print(f"Trained boosting target: {tname}, best_iter={m.best_iteration_}")


def predict_boost_split(X_np):
    """
    คืน dict:
      - prob / logits ที่ใช้ evaluate หรือเอาไปใช้ต่อได้
      - pred labels สำหรับดูผลลัพธ์
    """
    X_df = pd.DataFrame(X_np, columns=feature_cols)

    preds_prob = {}
    preds_label = {}

    for i, tname in enumerate(target_names):
        model = boost_models[tname]

        if tname == "tgt_candle_class":
            prob = model.predict_proba(X_df, num_iteration=model.best_iteration_)
            label = np.argmax(prob, axis=1)
        else:
            prob = model.predict_proba(X_df, num_iteration=model.best_iteration_)[:, 1]
            label = (prob >= 0.5).astype(int)

        preds_prob[tname] = prob
        preds_label[tname] = label

    return preds_prob, preds_label


def eval_boost(name, y_true, preds_label):
    print(f"\n[Boost {name}]")
    for i, tname in enumerate(target_names):
        yt = y_true[:, i]
        yp = preds_label[tname]

        acc = accuracy_score(yt, yp)
        if tname == "tgt_candle_class":
            f1 = f1_score(yt, yp, average="macro")
        else:
            f1 = f1_score(yt, yp)

        print(f"  {tname}: acc={acc:.4f}, f1={f1:.4f}")


# train
prob_train, pred_train = predict_boost_split(Xb_train)
prob_val, pred_val = predict_boost_split(Xb_val)
prob_test, pred_test = predict_boost_split(Xb_test)

eval_boost("train", yb_train, pred_train)
eval_boost("val", yb_val, pred_val)
eval_boost("test", yb_test, pred_test)

[LightGBM] [Info] Number of positive: 978, number of negative: 1094
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000257 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5887
[LightGBM] [Info] Number of data points in the train set: 2072, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.472008 -> initscore=-0.112086
[LightGBM] [Info] Start training from score -0.112086
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [26]:
# ============================
# Call 4: scale DL sequence inputs (StandardScaler) ให้พร้อมเทรน
# ============================

B, T, F = Xs_train.shape
print("Seq shapes (train/val/test):", Xs_train.shape, Xs_val.shape, Xs_test.shape)

scaler = StandardScaler()
scaler.fit(Xs_train.reshape(-1, F))


def scale_seq(Xs: np.ndarray) -> np.ndarray:
    Xflat = Xs.reshape(-1, F)
    Xflat = scaler.transform(Xflat)
    return Xflat.reshape(Xs.shape)


Xs_train_s = scale_seq(Xs_train)
Xs_val_s = scale_seq(Xs_val)
Xs_test_s = scale_seq(Xs_test)

print("Scaled seq shapes:", Xs_train_s.shape, Xs_val_s.shape, Xs_test_s.shape)
print("y shapes (targets multitask):", yd_train.shape, yd_val.shape, yd_test.shape)

Seq shapes (train/val/test): (2036, 51, 24) (436, 51, 24) (437, 51, 24)
Scaled seq shapes: (2036, 51, 24) (436, 51, 24) (437, 51, 24)
y shapes (targets multitask): (2036, 4) (436, 4) (437, 4)


In [27]:
# ============================
# Call 5: สร้าง DataLoader + โมเดล DL (TCN multitask classification)
# ============================


class SeqDataset(Dataset):
    def __init__(self, X_seq: np.ndarray, y: np.ndarray):
        self.X = torch.tensor(X_seq, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.int64)  # [B, 4] = 4 targets

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]


train_ds = SeqDataset(Xs_train_s, yd_train)
val_ds = SeqDataset(Xs_val_s, yd_val)
test_ds = SeqDataset(Xs_test_s, yd_test)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=128, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=128, shuffle=False)


class TCNBlock(nn.Module):
    def __init__(self, c_in, c_out, k=3, dilation=1, dropout=0.1):
        super().__init__()
        pad = (k - 1) * dilation
        self.net = nn.Sequential(
            nn.Conv1d(c_in, c_out, k, padding=pad, dilation=dilation),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Conv1d(c_out, c_out, k, padding=pad, dilation=dilation),
            nn.ReLU(),
            nn.Dropout(dropout),
        )
        self.down = nn.Conv1d(c_in, c_out, 1) if c_in != c_out else nn.Identity()

    def forward(self, x):
        y = self.net(x)
        y = y[..., : x.size(-1)]
        return y + self.down(x)


class MultiTaskTCN(nn.Module):
    def __init__(self, n_features: int, channels=(64, 64, 64)):
        super().__init__()
        layers = []
        c_in = n_features
        for i, c_out in enumerate(channels):
            layers.append(TCNBlock(c_in, c_out, dilation=2**i))
            c_in = c_out
        self.tcn = nn.Sequential(*layers)

        hidden_dim = channels[-1]
        # 1) breakout: high/low  (2 binary -> 2 logits)
        self.head_break = nn.Linear(hidden_dim, 2)
        # 2) volatility: high_vol (1 binary -> 1 logit)
        self.head_vol = nn.Linear(hidden_dim, 1)
        # 3) candle pattern: 3 classes
        self.head_candle = nn.Linear(hidden_dim, 3)

    def forward(self, x):
        # x: [B, T, F] -> [B, F, T]
        x = x.transpose(1, 2)
        z = self.tcn(x)  # [B, C, T]
        z_last = z[..., -1]  # [B, C]

        logits_break = self.head_break(z_last)  # [B, 2]
        logits_vol = self.head_vol(z_last).squeeze(-1)  # [B]
        logits_candle = self.head_candle(z_last)  # [B, 3]

        return logits_break, logits_vol, logits_candle


model = MultiTaskTCN(n_features=F).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

print(model)

MultiTaskTCN(
  (tcn): Sequential(
    (0): TCNBlock(
      (net): Sequential(
        (0): Conv1d(24, 64, kernel_size=(3,), stride=(1,), padding=(2,))
        (1): ReLU()
        (2): Dropout(p=0.1, inplace=False)
        (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,))
        (4): ReLU()
        (5): Dropout(p=0.1, inplace=False)
      )
      (down): Conv1d(24, 64, kernel_size=(1,), stride=(1,))
    )
    (1): TCNBlock(
      (net): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(2,))
        (1): ReLU()
        (2): Dropout(p=0.1, inplace=False)
        (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(2,))
        (4): ReLU()
        (5): Dropout(p=0.1, inplace=False)
      )
      (down): Identity()
    )
    (2): TCNBlock(
      (net): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(4,))
        (1): ReLU()
        (2): Dropout(p=0.1, inplace=Fa

In [28]:
# ============================
# Call 6: Train DL (early stopping by candle F1) + evaluate multi-task classification
# ============================

from sklearn.metrics import accuracy_score, f1_score

# ---- 1) สร้าง class weight สำหรับ candle ตาม distribution ใน train ----
candle_labels = yd_train[:, 3]  # คอลัมน์ 3 = tgt_candle_class
classes, counts = np.unique(candle_labels, return_counts=True)
freq = counts / counts.sum()
class_weights = 1.0 / (freq + 1e-8)  # class ยิ่งน้อย weight ยิ่งสูง
class_weights = class_weights / class_weights.mean()  # normalize ให้น้ำหนักเฉลี่ย = 1

class_weights_t = torch.tensor(class_weights, dtype=torch.float32, device=device)
print("Candle class weights:", dict(zip(classes, class_weights)))

# ---- 2) loss function + task weights ----
bce = nn.BCEWithLogitsLoss()
ce = nn.CrossEntropyLoss(weight=class_weights_t)

# เน้น breakout + candle มากกว่า vol (ซึ่งง่ายและแม่นอยู่แล้ว)
w_break = 0.7
w_vol = 0.2
w_candle = 1.0

best_f1_candle = -1.0
best_state = None
patience, wait = 10, 0
max_epochs = 100

for epoch in range(1, max_epochs + 1):
    # ---- train ----
    model.train()
    tr_losses = []
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)  # yb: [B,4]

        # แยก target
        y_break_high = yb[:, 0].float()
        y_break_low = yb[:, 1].float()
        y_vol = yb[:, 2].float()
        y_candle = yb[:, 3].long()

        logits_break, logits_vol, logits_candle = model(xb)

        loss_break_high = bce(logits_break[:, 0], y_break_high)
        loss_break_low = bce(logits_break[:, 1], y_break_low)
        loss_vol = bce(logits_vol, y_vol)
        loss_candle = ce(logits_candle, y_candle)

        # รวม loss แบบถ่วงน้ำหนัก
        loss = (
            w_break * (loss_break_high + loss_break_low)
            + w_vol * loss_vol
            + w_candle * loss_candle
        )

        opt.zero_grad()
        loss.backward()
        opt.step()
        tr_losses.append(loss.item())

    tr_loss = float(np.mean(tr_losses))

    # ---- val ----
    model.eval()
    va_losses = []
    ys_candle = []
    yp_candle = []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)

            y_break_high = yb[:, 0].float()
            y_break_low = yb[:, 1].float()
            y_vol = yb[:, 2].float()
            y_candle = yb[:, 3].long()

            logits_break, logits_vol, logits_candle = model(xb)

            loss_break_high = bce(logits_break[:, 0], y_break_high)
            loss_break_low = bce(logits_break[:, 1], y_break_low)
            loss_vol = bce(logits_vol, y_vol)
            loss_candle = ce(logits_candle, y_candle)

            loss = (
                w_break * (loss_break_high + loss_break_low)
                + w_vol * loss_vol
                + w_candle * loss_candle
            )

            va_losses.append(loss.item())

            # เก็บ candle สำหรับคำนวณ F1
            ys_candle.append(y_candle.cpu().numpy())
            yp_candle.append(np.argmax(logits_candle.cpu().numpy(), axis=1))

    va_loss = float(np.mean(va_losses))
    ys_candle = np.concatenate(ys_candle, axis=0)
    yp_candle = np.concatenate(yp_candle, axis=0)
    val_f1_candle = f1_score(ys_candle, yp_candle, average="macro")

    # ---- early stopping: ใช้ F1 ของ candle เป็นหลัก ----
    if val_f1_candle > best_f1_candle:
        best_f1_candle = val_f1_candle
        wait = 0
        best_state = {k: v.cpu() for k, v in model.state_dict().items()}
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping (by candle F1).")
            break

    if epoch % 5 == 0 or epoch == 1:
        print(
            f"Epoch {epoch:3d} | train_loss {tr_loss:.5f} | "
            f"val_loss {va_loss:.5f} | val_candle_f1 {val_f1_candle:.4f}"
        )

if best_state is not None:
    model.load_state_dict(best_state)


def eval_dl_split(loader, name: str):
    model.eval()
    ys = []
    yh_break_high = []
    yh_break_low = []
    yh_vol = []
    yh_candle = []

    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            yb_np = yb.numpy()  # keep y on CPU

            logits_break, logits_vol, logits_candle = model(xb)
            logits_break = logits_break.cpu().numpy()
            logits_vol = logits_vol.cpu().numpy()
            logits_candle = logits_candle.cpu().numpy()

            ys.append(yb_np)

            # binary: threshold 0.5
            yh_break_high.append(
                (1 / (1 + np.exp(-logits_break[:, 0])) >= 0.5).astype(int)
            )
            yh_break_low.append(
                (1 / (1 + np.exp(-logits_break[:, 1])) >= 0.5).astype(int)
            )
            yh_vol.append((1 / (1 + np.exp(-logits_vol)) >= 0.5).astype(int))
            yh_candle.append(np.argmax(logits_candle, axis=1))

    ys = np.concatenate(ys, axis=0)
    yh_break_high = np.concatenate(yh_break_high, axis=0)
    yh_break_low = np.concatenate(yh_break_low, axis=0)
    yh_vol = np.concatenate(yh_vol, axis=0)
    yh_candle = np.concatenate(yh_candle, axis=0)

    print(f"\n[DL {name}]")
    # 0: tgt_high_break, 1: tgt_low_break, 2: tgt_vol_high, 3: tgt_candle_class
    for idx, tname, yp in [
        (0, "tgt_high_break", yh_break_high),
        (1, "tgt_low_break", yh_break_low),
        (2, "tgt_vol_high", yh_vol),
        (3, "tgt_candle_class", yh_candle),
    ]:
        yt = ys[:, idx]
        acc = accuracy_score(yt, yp)
        if tname == "tgt_candle_class":
            f1 = f1_score(yt, yp, average="macro")
        else:
            f1 = f1_score(yt, yp)
        print(f"  {tname}: acc={acc:.4f}, f1={f1:.4f}")


eval_dl_split(train_loader, "train")
eval_dl_split(val_loader, "val")
eval_dl_split(test_loader, "test")

Candle class weights: {np.int64(0): np.float64(0.9391029537122482), np.int64(1): np.float64(1.0563076868022505), np.int64(2): np.float64(1.004589359485501)}
Epoch   1 | train_loss 2.15709 | val_loss 2.09013 | val_candle_f1 0.2551
Epoch   5 | train_loss 1.86543 | val_loss 1.93030 | val_candle_f1 0.3263
Epoch  10 | train_loss 1.69749 | val_loss 2.01259 | val_candle_f1 0.3186
Epoch  15 | train_loss 1.39393 | val_loss 2.26874 | val_candle_f1 0.3259
Epoch  20 | train_loss 1.09810 | val_loss 2.36685 | val_candle_f1 0.3362
Epoch  25 | train_loss 0.85727 | val_loss 2.87188 | val_candle_f1 0.3281
Epoch  30 | train_loss 0.65643 | val_loss 3.39637 | val_candle_f1 0.3263
Epoch  35 | train_loss 0.56789 | val_loss 3.47494 | val_candle_f1 0.3553
Early stopping (by candle F1).

[DL train]
  tgt_high_break: acc=0.9450, f1=0.9420
  tgt_low_break: acc=0.9352, f1=0.9335
  tgt_vol_high: acc=0.9784, f1=0.9701
  tgt_candle_class: acc=0.9558, f1=0.9560

[DL val]
  tgt_high_break: acc=0.6422, f1=0.5979
  tgt_l

In [29]:
# ============================
# Call 7: Save models (boosting + DL multitask) + scaler
# ============================

MODEL_DIR = DATA_DIR / "trained_models_multitask"
MODEL_DIR.mkdir(parents=True, exist_ok=True)

import joblib

# LightGBM models (dictionary: name -> model)
for name, m in boost_models.items():
    joblib.dump(m, MODEL_DIR / f"lgb_{name}.pkl")

# DL model (TCN multitask)
torch.save(model.state_dict(), MODEL_DIR / "tcn_multitask.pth")

# Scaler สำหรับ sequence features
joblib.dump(scaler, MODEL_DIR / "dl_scaler.pkl")

print("\n✔ Saved models to:", MODEL_DIR)
print(" - LightGBM (multitask heads):", list(boost_models.keys()))
print(" - MultiTask TCN state_dict: tcn_multitask.pth")
print(" - DL scaler: dl_scaler.pkl")


✔ Saved models to: /Users/thanaporn/Desktop/EURO_H1_AI/prepared_datasets/boosting_dl_multitask/trained_models_multitask
 - LightGBM (multitask heads): ['tgt_high_break', 'tgt_low_break', 'tgt_vol_high', 'tgt_candle_class']
 - MultiTask TCN state_dict: tcn_multitask.pth
 - DL scaler: dl_scaler.pkl
